In [13]:
import pandas as pd
import numpy as np
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from itertools import combinations_with_replacement
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [2]:
test = pd.read_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\Artificial\\Artificial_example_1_test.csv").drop(columns='Unnamed: 0')
train = pd.read_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\Artificial\\Artificial_example_1_train.csv").drop(columns='Unnamed: 0')
X_train = train.drop(columns='Target')
y_train= train['Target']
X_test = test.drop(columns='Target')
y_test = test['Target']

In [25]:
def MODEL(X_train, X_test, y_train, y_test, word="") :
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    print(word+' lr train_auc', round(roc_auc_score(y_train, lr.predict_proba(X_train)[:, 1]), 3))
    print(word+' lr test_auc', round(roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1]), 3))
    print(word+' lr train_f1', round(f1_score(y_train, lr.predict(X_train)), 3))
    print(word+' lr test_f1', round(f1_score(y_test, lr.predict(X_test)), 3))
    xg = xgb.XGBClassifier()
    xg.fit(X_train, y_train)
    print(word+' xgb train_auc', round(roc_auc_score(y_train, xg.predict_proba(X_train)[:, 1]), 3))
    print(word+' xgb test_auc', round(roc_auc_score(y_test, xg.predict_proba(X_test)[:, 1]), 3))
    print(word+' xgb train_f1', round(f1_score(y_train, xg.predict(X_train)), 3))
    print(word+' xgb test_f1', round(f1_score(y_test, xg.predict(X_test)), 3))


In [26]:
#ROW data
MODEL(X_train, X_test, y_train, y_test, "Row data")

Row data lr train_auc 0.872
Row data lr test_auc 0.869
Row data lr train_f1 0.458
Row data lr test_f1 0.455
Row data xgb train_auc 0.938
Row data xgb test_auc 0.928
Row data xgb train_f1 0.6
Row data xgb test_f1 0.584


In [4]:
#set up I-score function 
def I_score(X, y, influential) :
    I = 0
    test = 0
    total_mean = np.mean(y)
    data = X.copy()
    data['y'] = y
    m = len(influential)
    values_set = itertools.product([0, 1], repeat=m)
    for values in values_set :
        table = data
        for value, variable in zip(values, influential) :
            table = table[table[variable]==value]
        group_amount = len(table)
        test  += group_amount
        if  group_amount :
            sub_mean = sum(table['y'])/group_amount
            I += (group_amount**2)*((sub_mean-total_mean)**2)
    I /= len(data)
    return I

#set up thin out function
def retaining(X, y, influential) :
    standard = I_score(X, y, influential)
    while True :
        thin_influential = 0
        for se_influential in itertools.combinations(influential, len(influential)-1) :
            I = I_score(X, y, se_influential)
            if I > standard : 
                thin_influential = se_influential
                standard = I
        if thin_influential == 0 : return influential, standard
        influential = thin_influential

In [ ]:
# m_col = ['m=0', 'm=1', 'm=2', 'm=3', 'm=4', 'm=5', 'm=6', 'm=7', 'm=8', 'm=9', 'm=10', 'm=11', 'm=12']
# value_col = ['I0', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12']
# total_table = pd.DataFrame()
# for m, name, value in zip(range(1, 13), m_col[1:], value_col[1:]) :
#     unique_set = dict()
#     for combination in itertools.combinations(X_train.columns, m) :
#         thin, I = retaining(X_train, y_train, combination)
#         unique_set[thin] = round(I, 3) 
#     temp_table = pd.DataFrame({name : unique_set.keys(), value : unique_set.values()})
#     temp_table = temp_table.sort_values(value_col[m], ascending=False)
#     temp_table = temp_table.reset_index().drop(columns='index')
#     total_table = pd.concat([total_table, temp_table], axis=1)
# total_table.to_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\data\\Artificial\\Iscore.csv")

In [5]:
iscore_data = pd.read_csv("C:\\Users\\king0\\OneDrive\\鄧惠文專題\\data\\Artificial\\Iscore.csv").drop(columns="Unnamed: 0")

In [27]:
#PR
s = set()
lowest_I = 0
for m in range(2, 13) :
    for v, value in zip(iscore_data["m="+str(m)], iscore_data["I"+str(m)]) :
        if isinstance(v, str) :
            if len(eval(v)) > 1 and value > lowest_I :
                s.add(eval(v))
                

new_X_train = pd.DataFrame()
new_X_test = pd.DataFrame()
for variables in s :
    new_X_train[variables] = X_train[list(variables)].product(axis=1)
    new_X_test[variables] = X_test[list(variables)].product(axis=1)

MODEL(new_X_train, new_X_test, y_train, y_test, "PR")

PR lr train_auc 0.932
PR lr test_auc 0.929
PR lr train_f1 0.585
PR lr test_f1 0.582
PR xgb train_auc 0.936
PR xgb test_auc 0.928
PR xgb train_f1 0.595
PR xgb test_f1 0.585


In [28]:
#PG 337
s = set()
lowest_I = 336
for m in range(2, 13) :
    for v, value in zip(iscore_data["m="+str(m)], iscore_data["I"+str(m)]) :
        if isinstance(v, str) :
            if len(eval(v)) > 1 and value > lowest_I :
                s.add(eval(v))
                

new_X_train = pd.DataFrame()
new_X_test = pd.DataFrame()
for variables in s :
    new_X_train[variables] = X_train[list(variables)].product(axis=1)
    new_X_test[variables] = X_test[list(variables)].product(axis=1)

MODEL(new_X_train, new_X_test, y_train, y_test, "PR337")

PR337 lr train_auc 0.93
PR337 lr test_auc 0.929
PR337 lr train_f1 0.585
PR337 lr test_f1 0.582
PR337 xgb train_auc 0.93
PR337 xgb test_auc 0.929
PR337 xgb train_f1 0.585
PR337 xgb test_f1 0.582


In [34]:
#Quadratic 
combi = []
i_score = []
variables = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12']

combination = list(combinations_with_replacement(variables, 2))
for i in variables :
    combination.append(tuple([i]))
                                                 
for v in combination :
    combi.append(v)
    i_score.append(I_score(X_train, y_train, list(v)))
table = pd.DataFrame({'variables':combi, 'iscore': i_score})
table = table.sort_values('iscore', ascending=False)

#Quadratic 10% section
topv = table.iloc[:9]['variables'].reset_index(drop=True)
new_X_train = pd.DataFrame()
new_X_test = pd.DataFrame()
for v in topv :
    new_X_train[v] = X_train[list(v)].product(axis=1)
    new_X_test[v] = X_test[list(v)].product(axis=1)

MODEL(new_X_train, new_X_test, y_train, y_test, "Quadratic 10%")

print()
#Quadratic section
topv = table['variables'].reset_index(drop=True)

new_X_train = pd.DataFrame()
new_X_test = pd.DataFrame()
for v in topv :
    new_X_train[v] = X_train[list(v)].product(axis=1)
    new_X_test[v] = X_test[list(v)].product(axis=1)

MODEL(new_X_train, new_X_test, y_train, y_test, "Quadratic")
table

Quadratic 10% lr train_auc 0.918
Quadratic 10% lr test_auc 0.916
Quadratic 10% lr train_f1 0.637
Quadratic 10% lr test_f1 0.627
Quadratic 10% xgb train_auc 0.931
Quadratic 10% xgb test_auc 0.929
Quadratic 10% xgb train_f1 0.585
Quadratic 10% xgb test_f1 0.582

Quadratic lr train_auc 0.932
Quadratic lr test_auc 0.929
Quadratic lr train_f1 0.62
Quadratic lr test_f1 0.612
Quadratic xgb train_auc 0.939
Quadratic xgb test_auc 0.928
Quadratic xgb train_f1 0.608
Quadratic xgb test_f1 0.58


,variables,iscore
45,"(x5, x8)",895.085866
82,"(x5,)",608.878444
42,"(x5, x5)",608.878444
85,"(x8,)",589.346303
63,"(x8, x8)",589.346303
...,...,...
68,"(x9, x9)",0.007792
50,"(x6, x6)",0.006298
83,"(x6,)",0.006298
77,"(x12, x12)",0.004839


In [32]:
#Cubic 
combi = []
i_score = []
variables = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12']

combination = list(combinations_with_replacement(variables, 3))
combination += list(combinations_with_replacement(variables, 2))
for i in variables :
    combination.append(tuple([i]))
                                                 
for v in combination :
    combi.append(v)
    i_score.append(I_score(X_train, y_train, list(v)))
table = pd.DataFrame({'variables':combi, 'iscore': i_score})
table = table.sort_values('iscore', ascending=False)

#Cubic 10% Section
topv = table.iloc[:45]['variables'].reset_index(drop=True)
new_X_train = pd.DataFrame()
new_X_test = pd.DataFrame()
for v in topv :
    new_X_train[v] = X_train[list(v)].product(axis=1)
    new_X_test[v] = X_test[list(v)].product(axis=1)
MODEL(new_X_train, new_X_test, y_train, y_test, "Cubic 10%")
print()
#Cubic section
topv = table['variables'].reset_index(drop=True)
new_X_train = pd.DataFrame()
new_X_test = pd.DataFrame()
for v in topv :
    new_X_train[v] = X_train[list(v)].product(axis=1)
    new_X_test[v] = X_test[list(v)].product(axis=1)
MODEL(new_X_train, new_X_test, y_train, y_test, "Cubic")

Cubic 10% lr train_auc 0.931
Cubic 10% lr test_auc 0.93
Cubic 10% lr train_f1 0.585
Cubic 10% lr test_f1 0.582
Cubic 10% xgb train_auc 0.934
Cubic 10% xgb test_auc 0.929
Cubic 10% xgb train_f1 0.59
Cubic 10% xgb test_f1 0.586

Cubic lr train_auc 0.934
Cubic lr test_auc 0.928
Cubic lr train_f1 0.586
Cubic lr test_f1 0.584
Cubic xgb train_auc 0.939
Cubic xgb test_auc 0.928
Cubic xgb train_f1 0.605
Cubic xgb test_f1 0.578
